#### Section 1.4.1 Voltage Dividers
Taken from "Switch-Mode Power Supplies" by Christophe P. Basso

#### Imports

In [16]:
import os
from pathlib import Path
import copy
from typing import TypeAlias
import numpy as np
import numpy.typing as npt
import py4spice as spi
from py4spice import globals_types

#### Path to Ngspice and the project

In [17]:
# Path to NGSPICE
NGSPICE_EXE = Path("/usr/bin/ngspice")

# Path to my project
PROJ_PATH = Path("/workspaces/sw_pwr_book_sim/circuits/sec_1_04_01_dividers")

#### Prepare simulation results directory

In [18]:
# Create sim_results directory if doesn't exist
RESULTS_PATH = PROJ_PATH / "sim_results"
RESULTS_PATH.mkdir(parents=True, exist_ok=True)

#### Netlists defining

In [19]:
# Create netlists directory and files if they don't exist
NETLISTS_PATH = PROJ_PATH / "netlists"
NETLISTS_PATH.mkdir(parents=True, exist_ok=True)  # Create the directory

# Create title.cir with content only if it doesn't exist
TITLE_FILENAME = NETLISTS_PATH / "title.cir"
if not TITLE_FILENAME.exists():
    with open(TITLE_FILENAME, "w") as f:
        f.write("* First line in netlist must be a comment")
title = spi.Netlist(NETLISTS_PATH / TITLE_FILENAME)

# These netlist objects will be combined and a top netlist will be created later
dut = spi.Netlist(NETLISTS_PATH / "dut.cir")
load = spi.Netlist(NETLISTS_PATH / "load.cir")
stimulus = spi.Netlist(NETLISTS_PATH / "stimulus.cir")
supplies = spi.Netlist(NETLISTS_PATH / "supplies.cir")

# these are two files the program will create later
TOP_FILENAME = NETLISTS_PATH / "top.cir"
CONTROL_FILENAME = NETLISTS_PATH / "control.cir"

Convert easyeda_dut.cir to dut.cir

In [20]:
easyeda = spi.Netlist(NETLISTS_PATH / "easyeda_dut.cir")
easyeda.del_line_starts_with("*") # remove first line (comment)
easyeda.del_line_starts_with("VEASYEDA") # remove fake voltage source
easyeda.del_line_starts_with(".TRAN 1M 3.1415") # remove fake transient analysis
dut = easyeda
dut.write_to_file(NETLISTS_PATH / "dut.cir") # this is now the dut (device under test)

#### Create signal vectors

In [21]:
VEC_ALL = spi.Vectors("ALL")
VEC_IN_OUT = spi.Vectors(["IN", "OUT1", "OUT2"])
VEC_ALL_EXPANDED = spi.Vectors(
    [
        "IN",
        "OUT1",
        "OUT1_MEAS",
        "OUT2",
        "OUT2_MEAS",
        "VEE",
        "VEE#BRANCH",
        "VIN#BRANCH",
        "VMEAS1#BRANCH",
        "VMEAS2#BRANCH",
    ]
)
VEC_VIN = spi.Vectors("IN")
VEC_IIN = spi.Vectors("VIN#BRANCH")
VEC_VOUT1 = spi.Vectors("OUT1")
VEC_VOUT2 = spi.Vectors("OUT2")
VEC_IOUT1 = spi.Vectors("VMEAS1#BRANCH")
VEC_IOUT2 = spi.Vectors("VMEAS2#BRANCH")

#### Define analyses

In [22]:
list_of_analyses: list[spi.Analyses] = [] # create empty list. Next sections define

# 1st analysis: operating point
# for this simulation, we'll have only one analysis
op_cmd = "op"
op1 = spi.Analyses("op1", "op", op_cmd, VEC_ALL, RESULTS_PATH)
list_of_analyses.append(op1)

#### Create control section

In [23]:
my_control = spi.Control()  # create 'my_control' object
my_control.insert_lines(["listing"])  # cmd to list out netlist
for analysis in list_of_analyses:  # statements for all analyses
    my_control.insert_lines(analysis.lines_for_cntl())
spi.print_section("Control File", my_control)  # print out contents
my_control.content_to_file(CONTROL_FILENAME)  # creat the actual file
control = spi.Netlist(NETLISTS_PATH / CONTROL_FILENAME) # create netlist object


--- Control File ---
.control
* Timestamp: Sat Feb 10 19:24:15 2024
set wr_singlescale  $ makes one x-axis for wrdata
set wr_vecnames     $ puts names at top of columns
listing
op
print line ALL > /workspaces/sw_pwr_book_sim/circuits/sec_1_04_01_dividers/sim_results/op1.txt
quit
.endc
--------------------



#### Simulate

In [24]:
# Combine the netlists and write out into one top netlist, ready to simulate
top = title + dut + load + supplies + stimulus + control

top.write_to_file(TOP_FILENAME)
spi.print_section("top netlist", top)

# prepare simulate object, print out command, and simulate
sim1 = spi.Simulate(NGSPICE_EXE, TOP_FILENAME)
spi.print_section("Ngspice Command", sim1)
sim1.run() # run the Ngspice simulation


--- top netlist ---
* Two voltage dividers
R1 IN OUT1 140
R2 IN OUT2 132
RLOAD1 OUT1 VEE 100
RLOAD2 OUT2 VEE 50
VEE VEE 0 0
VIN IN 0 12
.control
* Timestamp: Sat Feb 10 19:24:15 2024
set wr_singlescale  $ makes one x-axis for wrdata
set wr_vecnames     $ puts names at top of columns
listing
op
print line ALL > /workspaces/sw_pwr_book_sim/circuits/sec_1_04_01_dividers/sim_results/op1.txt
quit
.endc
-------------------


--- Ngspice Command ---
/usr/bin/ngspice -b /workspaces/sw_pwr_book_sim/circuits/sec_1_04_01_dividers/netlists/top.cir
-----------------------


Note: No compatibility mode selected!


Circuit: * two voltage dividers

	* two voltage dividers

     2 : .global gnd
     3 : r1 in out1 140
     4 : r2 in out2 132
     5 : rload1 out1 vee 100
     6 : rload2 out2 vee 50
     7 : vee vee 0 0
     8 : vin in 0 12
     9 : .control
    11 : set wr_singlescale
    12 : set wr_vecnames
    13 : listing
    14 : op
    15 : print line all > /workspaces/sw_pwr_book_sim/circuits/se

#### Gather simulation results

In [51]:
# Path and name of result file
RESULTS_FILENAME = RESULTS_PATH / f"{list_of_analyses[0].name}.txt"

# Create signals object from the result file
op1 = spi.Signals.from_spice_table(RESULTS_FILENAME)
print(VEC_ALL_EXPANDED.list_out)
fred = VEC_ALL_EXPANDED.list_out
print(fred)
print(op1)
# spi.print_section("Operating Point", op1_values)

['IN', 'OUT1', 'OUT1_MEAS', 'OUT2', 'OUT2_MEAS', 'VEE', 'VEE#BRANCH', 'VIN#BRANCH', 'VMEAS1#BRANCH', 'VMEAS2#BRANCH']
['IN', 'OUT1', 'OUT1_MEAS', 'OUT2', 'OUT2_MEAS', 'VEE', 'VEE#BRANCH', 'VIN#BRANCH', 'VMEAS1#BRANCH', 'VMEAS2#BRANCH']
Signals(x_data=[0.0], y_data_list=[array(12.), array(5.), array(3.296703), array(0.), array(0.1159341), array(-0.115934)], x_label='', y_labels=['in', 'out1', 'out2', 'vee', 'vee#branch', 'vin#branch']
